## A4

In [1]:
from pyspark.sql import SparkSession

# New API
spark_session = SparkSession\
        .builder\
        .master("spark://haol-spark:7077") \
        .appName("Question_A4")\
        .getOrCreate()

# Old API (RDD)
spark_context = spark_session.sparkContext

In [2]:
eng = spark_context.textFile("/home/ubuntu/LDSA/ass_2/swe_eng/europarl-v7.sv-en.en")\
.pipe("/home/ubuntu/LDSA/ass_2/tools/tokenizer.perl -l en")
swe = spark_context.textFile("/home/ubuntu/LDSA/ass_2/swe_eng/europarl-v7.sv-en.sv")\
.pipe("/home/ubuntu/LDSA/ass_2/tools/tokenizer.perl -l sv")

In [3]:
engIndex = eng.zipWithIndex()
sweIndex = swe.zipWithIndex()

In [4]:
engIndexInv = engIndex.map(lambda x: (x[1], x[0]))
sweIndexInv = sweIndex.map(lambda x: (x[1], x[0]))

In [5]:
joined = sweIndexInv.join(engIndexInv)

In [6]:
joined.take(2)

[(0, ('Återupptagande av sessionen', 'Resumption of the session')),
 (1835010,
  ('Jag hoppas att han tar sig litet tid för att gå in på det .',
   'I hope that he will take a little time to address this issue before we close .'))]

In [10]:
words = joined.map(lambda x: (x[0], (x[1][0].split(" "), x[1][1].split(" "))))

In [11]:
words.take(2)

[(0,
  (['Återupptagande', 'av', 'sessionen'],
   ['Resumption', 'of', 'the', 'session'])),
 (1835010,
  (['Jag',
    'hoppas',
    'att',
    'han',
    'tar',
    'sig',
    'litet',
    'tid',
    'för',
    'att',
    'gå',
    'in',
    'på',
    'det',
    '.'],
   ['I',
    'hope',
    'that',
    'he',
    'will',
    'take',
    'a',
    'little',
    'time',
    'to',
    'address',
    'this',
    'issue',
    'before',
    'we',
    'close',
    '.']))]

In [24]:
wordsFiltered = words.filter(lambda x: not " " == x[1][1] or not " " == x[1][0])\
.filter(lambda x: len(x[1][1])<5)\
.filter(lambda x: len(x[1][0])<5)\
.filter(lambda x: len(x[1][0])==len(x[1][1]))

In [25]:
wordsFiltered1 = wordsFiltered.flatMap(lambda x: x[1][0])
wordsFiltered2 = wordsFiltered.flatMap(lambda x: x[1][1])
dictionary = wordsFiltered1.zip(wordsFiltered2)

In [26]:
wordsFiltered1.count()

40626

In [27]:
wordsFiltered2.count()

40626

In [28]:
dictionary.take(2)

[('Vi', 'We'), ('måste', 'must')]

In [31]:
dictionary.map(lambda x: (x, 1))\
.reduceByKey(lambda x, y: x+y)\
.sortBy(lambda x: x[1], False)

PythonRDD[48] at RDD at PythonRDD.scala:48

In [32]:
dictionary.take(20)

[('Vi', 'We'),
 ('måste', 'must'),
 ('agera', 'act'),
 ('.', '.'),
 ('(', '('),
 ('1', '1'),
 (')', ')'),
 ('(', '('),
 ('Applåder', 'Applause'),
 (')', ')'),
 ('.', ''),
 ('.', ''),
 ('Ändringsförslag', 'On'),
 ('nr', 'Amendment'),
 ('.', 'No.'),
 ('1', '1'),
 ('Luftfartsavtal', 'EU-US'),
 ('EU-Förenta', 'air'),
 ('staterna', 'agreement'),
 ('(', '(')]

In [33]:
# release the cores for another application!
spark_context.stop()